In [ ]:
import pandas as pd
import sys
import numpy as np
from itertools import combinations
pd.set_option('display.max_rows', 100)

In [ ]:
sys.path.append('..\\packages')

In [ ]:
from suppression_check import DataAnonymizer

# CSV file with PreRedacted Data

In [ ]:
file_path = r'..\data\UserData.csv'

# Load the CSV file into a DataFrame

In [ ]:
df_data = pd.read_csv(file_path)

# Instantiate the anonymizer

In [ ]:
sensitive_list = ['Subgroup1', 'Subgroup2']

In [ ]:
frequency_value = 'Counts'

In [ ]:
anonymizer = DataAnonymizer(df_data, sensitive_columns=sensitive_list, frequency=frequency_value)

In [ ]:
df_log = anonymizer.create_log()

In [ ]:
df_log.to_csv('Log.csv')

In [ ]:
df_less_than_threshold = anonymizer.less_than_threshold_not_zero()

In [ ]:
df_less_than_threshold.to_csv('LessThanThreshold.csv')

In [ ]:
sensitive_combinations = sorted([combo for i in range(1, len(sensitive_list) + 1) for combo in combinations(sensitive_list, i)], key=len, reverse=True)

In [ ]:
df_primary_redacted = df_less_than_threshold[df_less_than_threshold['RedactBinary'] == 1]

In [ ]:
df_primary_redacted['PrimaryRedacted'] = 1

In [ ]:
for sensitive_combination in sensitive_combinations:
    list_combination = list(sensitive_combination)
    if list_combination != sensitive_list:
        string_combination = ''.join(list_combination)
        df_primary_merge = df_primary_redacted[['Grouping'] + list_combination + ['PrimaryRedacted']]
        df_primary = df_less_than_threshold.merge(df_primary_merge, on=['Grouping'] + list_combination, how='left')
        mask = (df_primary['PrimaryRedacted'] == 1) & (df_primary["MinimumValue" + string_combination] == df_primary[frequency_value])
        df_less_than_threshold.loc[mask, 'RedactBinary'] = 1
        df_less_than_threshold.loc[mask, 'Redact'] = 'Secondary Suppression'

In [ ]:
df_less_than_threshold.to_csv('OneCountRedact.csv')

In [ ]:
df_one_count = anonymizer.one_count_redacted()

In [ ]:
display(df_one_count)

In [ ]:
df_sum_redact = anonymizer.sum_redact()

In [ ]:
display(df_sum_redact)

In [ ]:
df_zero = anonymizer.one_redact_zero()

In [ ]:
display(df_zero)

In [ ]:
df_cross_suppression = anonymizer.cross_suppression()

In [ ]:
display(df_cross_suppression)

In [ ]:
df_apply_log = anonymizer.apply_log()

In [ ]:
display(df_apply_log)

In [ ]:
df_apply_log.to_csv('MergedTotals.csv')